In [1]:
import pathlib
# BASE_DIR will be like '/home/jovyan/DemoExample/'
BASE_DIR = pathlib.Path().absolute()
print(f"Working dir: {BASE_DIR}")

Working dir: /home/jovyan/greenAI_gpt


@Sayankotor

In [2]:
# !pip install transformers datasets

In [3]:
# !pip install wandb

In [ ]:
import torch
from torch.nn import functional as F
import numpy as np
# from transformers import BertModel, BertConfig, BertTokenizer, BertLMHeadModel, BertTokenizerFast
from transformers import *
from datasets import load_dataset

import os
import sys
from tqdm import tqdm
import random
import math



In [ ]:
wandb.login()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

## References

https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset - here are examples of overtaking multiprocessing in iterable datasets

## Openwebtext

In [5]:
import os
import re
from itertools import chain

import datasets


_CITATION = """\
@misc{Gokaslan2019OpenWeb,
  title={OpenWebText Corpus},
  author={Aaron Gokaslan*, Vanya Cohen*, Ellie Pavlick, Stefanie Tellex},
  howpublished{\\url{http://Skylion007.github.io/OpenWebTextCorpus}},
  year={2019}
}
"""

_DESCRIPTION = """\
An open-source replication of the WebText dataset from OpenAI.
"""

_URL = "https://zenodo.org/record/3834942/files/openwebtext.tar.xz"


class Openwebtext(datasets.GeneratorBasedBuilder):
    """The Open WebText dataset."""

    BUILDER_CONFIGS = [
        datasets.BuilderConfig(
            name="plain_text",
            description="Plain text",
            version=datasets.Version("1.0.0"),
        )
    ]

    def _info(self):
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=datasets.Features({"text": datasets.Value("string")}),
            homepage="https://skylion007.github.io/OpenWebTextCorpus/",
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        dl_dir = dl_manager.download_and_extract(_URL)
        owt_dir = os.path.join(dl_dir, "openwebtext")
        subset_xzs = [
            os.path.join(owt_dir, file_name)
            for file_name in sorted(os.listdir(owt_dir))
            if file_name.endswith("xz")  # filter out ...xz.lock
        ]
        ex_dirs = dl_manager.extract(subset_xzs, num_proc=round(os.cpu_count() * 0.75))
        nested_txt_files = [
            [
                os.path.join(ex_dir, txt_file_name)
                for txt_file_name in sorted(os.listdir(ex_dir))
                if txt_file_name.endswith("txt")
            ]
            for ex_dir in ex_dirs
        ]
        txt_files = chain(*nested_txt_files)
        return [
            datasets.SplitGenerator(name=datasets.Split.TRAIN, gen_kwargs={"txt_files": txt_files}),
        ]

    def _generate_examples(self, txt_files):
        """Yields examples."""
        for idx, filepath in enumerate(txt_files):
            with open(filepath, encoding="utf-8") as f:
                yield idx, {"text": re.sub("\n\n\n+", "\n\n", f.read()).strip()}


2022-06-13 22:31:05.171484: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


# Datasets

preprocess openwebtext:
1. curl https://zenodo.org/record/3834942/files/openwebtext.tar.xz -o openwebtext.tar.xz
2. in cmd:

for i in *.xz; do tar -xf "$i" -C "/notebook/greenAI/owt_files/openwebtext/texts" ; done 

In [14]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    print (dirName)
    allFiles = list()
    # Iterate over all the entries
    i = 0
    for entry in listOfFile:
        # Create full path
        #fullPath = os.path.join(dirName, entry)
        if (i%50000 == 0):
            print (i)
        i += 1
        # If entry is a directory then get the list of files in this directory 
        #if os.path.isdir(fullPath):
            #allFiles = allFiles + getListOfFiles(fullPath)
        #else:
            #allFiles.append(fullPath)
    print (i)            
    return allFiles



In [22]:
import pathlib
# BASE_DIR will be like '/home/jovyan/DemoExample/'
BASE_DIR = pathlib.Path().absolute()
print(f"Working dir: {BASE_DIR}")

Working dir: /home/jovyan/greenAI_gpt


In [18]:
import os
filelist = getListOfFiles('/home/jovyan/greenAI_gpt/openwebtext/text1')
len(filelist)

/home/jovyan/greenAI_gpt/openwebtext/text1
0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1648619


0

In [19]:
len(filelist)

0

In [14]:
# ls "../data/sci-hub/txt/00000000/00000000/10.1002" | head -10

In [11]:
class FileListIterator:
    def __init__(self, filelist):
        self.filelist = filelist
        self.nfiles = len(self.filelist)
        
    def __iter__(self):
        # self.lines = 0
        self.fileidx = 0
        self.fin = open(self.filelist[self.fileidx], "r")
        while True:
            line = self.fin.readline()
            # self.lines += 1
            if line == "":
                # reached EOF
                # print('reached eof of file', self.fileidx, self.nfiles, self.lines)
                # self.lines = 0
                self.fin.close()
                self.fileidx += 1
                if self.fileidx > self.nfiles - 1:
                    # end of filelist
                    # print('reached end of filelist', self.fileidx)
                    break
                else:
                    self.fin = open(self.filelist[self.fileidx], "r")
                    line = self.fin.readline()
                    yield line.strip("\n")
            else:
                yield line.strip("\n")
                

In [12]:
class FileListIteratorMultiproc:
    def __init__(self, filelist):
        self.filelist = filelist
        self.nfiles = len(self.filelist)
        
    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        print ("worker_info it", worker_info)
        # self.lines = 0
        self.fileidx = 0
        self.shift = 1
        if worker_info is not None:
            #per_worker = int(math.ceil((self.end - self.start) / float(worker_info.num_workers)))
            worker_id = worker_info.id
            self.fileidx = self.fileidx + worker_id
            self.shift = worker_info.num_workers
            print ("multiproc")
            print ("worker id", worker_id)
            print ("self.fileidx ", self.filelist[self.fileidx])
            print ("shift", self.shift)
            print ("next", self.filelist[self.fileidx + self.shift])
            print ("\n\n")
            
        self.fin = open(self.filelist[self.fileidx], "r")
        # single-process data loading, return the full iterator
        while True:
            line = self.fin.readline()
            # self.lines += 1
            if line == "":
                # reached EOF
                # print('reached eof of file', self.fileidx, self.nfiles, self.lines)
                # self.lines = 0
                self.fin.close()
                self.fileidx += self.shift
                if self.fileidx > self.nfiles - worker_info.num_workers:
                    # end of filelist
                    # print('reached end of filelist', self.fileidx)
                    break
                else:
                    self.fin = open(self.filelist[self.fileidx], "r")
                    line = self.fin.readline()
                    yield line.strip("\n")
            else:
                yield line.strip("\n")

In [35]:
class FileListDatasetWithMaxFiles(torch.utils.data.IterableDataset):
    def __init__(self, iterator, tokenizer, seq_len, filelist = '', max_files = 1):
        self._iterator = iterator
        self._tokenizer =tokenizer
        self.seq_len = seq_len
        self.filelist = filelist
        self.max_files = max_files
        
    
    @classmethod
    def from_filelist(cls, filelist, tokenizer, seq_len):
        worker_info = torch.utils.data.get_worker_info()
        print ("worker_info", worker_info)
        iterator = FileListIterator(filelist=filelist)
        return cls(
            iterator=iterator,
            tokenizer=tokenizer,
            seq_len=seq_len,
            filelist = filelist
        )
    
    def __iter__(self):
        """
            Yields (List[int])
        """
        assert torch.utils.data.get_worker_info() is None
        ids = []
        global_text = ''
        nproc_files = 0
        for text in self._iterator:
            while (nproc_files < self.max_files):
                global_text += text
                nproc_files += 1
            #print (self.seq_len)
            #print (len(global_text))
            #print (global_text)
            ids.extend(self._tokenizer.encode(global_text))
            while len(ids) >= self.seq_len+1:
                print ("yield")
                print (len(ids[:self.seq_len]))
                #print (ids[:self.seq_len])
                print ("\n\n\n")
                yield {"input_ids": ids[:self.seq_len],
                       "labels": ids[:self.seq_len]}
                ids = ids[self.seq_len:]

    @classmethod
    def collate_fn(cls, item):
        """Collate function for DataLoader
        Args:
            item (List[dict[str, List[int]]])
        Returns:
            (dict[str, torch.Tensor]):
        """
        keys = item[0].keys()
        dic = {
            key: torch.tensor([x[key] for x in item])
            for key in keys
        }
        return dic
    
    def __len__(self):
        return len(self.filelist)

In [43]:
class FileListDataset(torch.utils.data.IterableDataset):
    def __init__(self, iterator, tokenizer, seq_len, filelist = '', max_files = 1):
        self._iterator = iterator
        self._tokenizer =tokenizer
        self.seq_len = seq_len
        self.filelist = filelist
        self.max_files = max_files
        
    
    @classmethod
    def from_filelist(cls, filelist, tokenizer, seq_len):
        worker_info = torch.utils.data.get_worker_info()
        print ("worker_info", worker_info)
        iterator = FileListIterator(filelist=filelist)
        return cls(
            iterator=iterator,
            tokenizer=tokenizer,
            seq_len=seq_len,
            filelist = filelist
        )
    
    def __iter__(self):
        """
            Yields (List[int])
        """
        assert torch.utils.data.get_worker_info() is None
        ids = []
        for text in self._iterator:
            ids.extend(self._tokenizer.encode(text))
            while len(ids) >= self.seq_len+1:
                print ("yield")
                print (len(ids[:self.seq_len]))
                #print (ids[:self.seq_len])
                print ("\n\n\n")
                yield {"input_ids": ids[:self.seq_len],
                       "labels": ids[:self.seq_len]}
                ids = ids[self.seq_len:]

    @classmethod
    def collate_fn(cls, item):
        """Collate function for DataLoader
        Args:
            item (List[dict[str, List[int]]])
        Returns:
            (dict[str, torch.Tensor]):
        """
        keys = item[0].keys()
        dic = {
            key: torch.tensor([x[key] for x in item])
            for key in keys
        }
        return dic
    
    def __len__(self):
        return len(self.filelist)

In [24]:
class ShuffleDataset(torch.utils.data.IterableDataset):
    def __init__(self, dataset, buffer_size, seed=None):
        super().__init__()
        self.dataset = dataset
        self.buffer_size = buffer_size
        self.rns = np.random.RandomState(seed)

    def __iter__(self):
        shufbuf = []
        try:
            dataset_iter = iter(self.dataset)
            for i in range(self.buffer_size):
                shufbuf.append(next(dataset_iter))
        except:
            self.buffer_size = len(shufbuf)

        try:
            while True:
                try:
                    # yielding random element and replacing it with new item from dataset
                    item = next(dataset_iter)
                    evict_idx = self.rns.randint(0, self.buffer_size - 1)
                    yield shufbuf[evict_idx]
                    shufbuf[evict_idx] = item
                except StopIteration:
                    break
            while len(shufbuf) > 0:  # yielding the remaining items in shuffle buff
                yield shufbuf.pop()
        except GeneratorExit:
            pass

In [15]:
from transformers import GPT2Tokenizer
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

loading file https://huggingface.co/gpt2/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/684fe667923972fb57f6b4dcb61a3c92763ad89882f3da5da9866baf14f2d60f.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f
loading file https://huggingface.co/gpt2/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/c0c761a63004025aeadd530c4c27b860ec4ecbe8a00531233de21d865a402598.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/gpt2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/gpt2/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/gpt2/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb1993821

In [36]:
train_dataset = FileListDataset.from_filelist(filelist=filelist[:200], 
                                              tokenizer=tokenizer, 
                                              seq_len=512)

worker_info None


In [37]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [38]:
train_dataloader = DataLoader(train_dataset, batch_size=4, collate_fn=FileListDataset.collate_fn, drop_last=True)

## in one process it is ok

In [41]:
from tqdm import tqdm
epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=True)
for step, batch in enumerate(epoch_iterator):
    inputs, labels = (batch["input_ids"], batch["input_ids"])
    print (len(batch["input_ids"]))
    print (len(batch["input_ids"][0]))
    print (batch["input_ids"])
    print ("\n\n\n")
    

yield
512




yield
512




yield
512




yield
512




4
512
tensor([[ 5189,   262,  4153,  ...,   379,   262,  5776],
        [   11,   517,   621,  ..., 20736,  3765,    11],
        [ 1390,  7027,   588,  ...,   461, 14192,   326],
        [ 5252,  2807,   338,  ...,   351,   663,  5909]])




yield
512




yield
512




yield
512




yield
512




4
512
tensor([[14072,   286,  3056,  ...,    11,   290, 21622],
        [   13,  2773,  1542,  ...,   262,  2057,  4983],
        [  379,   262,  5776,  ..., 11023,   282,    13],
        [ 5189,   262,  4153,  ...,   379,   262,  5776]])




yield
512




yield
512




yield
512




yield
512




4
512
tensor([[   11,   517,   621,  ..., 20736,  3765,    11],
        [ 1390,  7027,   588,  ...,   461, 14192,   326],
        [ 5252,  2807,   338,  ...,   351,   663,  5909],
        [14072,   286,  3056,  ...,    11,   290, 21622]])




yield
512




yield
512




yield
512




yield
512




4
512
tensor([[   13,  2773,  1542,  ...,   262

# multipropcessing

In [28]:
class FileListIteratorMultiproc:
    def __init__(self, filelist):
        self.filelist = filelist
        self.nfiles = len(self.filelist)
        
    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        print ("worker_info it", worker_info)
        # self.lines = 0
        self.fileidx = 0
        self.shift = 1
        if worker_info is not None:
            #per_worker = int(math.ceil((self.end - self.start) / float(worker_info.num_workers)))
            worker_id = worker_info.id
            self.fileidx = self.fileidx + worker_id
            self.shift = worker_info.num_workers
            print ("multiproc")
            print ("worker id", worker_id)
            print ("self.fileidx ", self.filelist[self.fileidx])
            print ("shift", self.shift)
            print ("next", self.filelist[self.fileidx + self.shift])
            print ("\n\n")
            
        self.fin = open(self.filelist[self.fileidx], "r")
        # single-process data loading, return the full iterator
        while True:
            line = self.fin.readline()
            # self.lines += 1
            if line == "":
                # reached EOF
                # print('reached eof of file', self.fileidx, self.nfiles, self.lines)
                # self.lines = 0
                self.fin.close()
                self.fileidx += self.shift
                if self.fileidx > self.nfiles - worker_info.num_workers:
                    # end of filelist
                    # print('reached end of filelist', self.fileidx)
                    break
                else:
                    self.fin = open(self.filelist[self.fileidx], "r")
                    line = self.fin.readline()
                    yield line.strip("\n")
            else:
                yield line.strip("\n")

In [29]:
class FileListDataset(torch.utils.data.IterableDataset):
    def __init__(self, iterator, tokenizer, seq_len, filelist = ''):
        self._iterator = iterator
        self._tokenizer =tokenizer
        self.seq_len = seq_len
        self.filelist = filelist
        
    
    @classmethod
    def from_filelist(cls, filelist, tokenizer, seq_len):
        worker_info = torch.utils.data.get_worker_info()
        print ("worker_info", worker_info)
        iterator = FileListIteratorMultiproc(filelist=filelist)
        return cls(
            iterator=iterator,
            tokenizer=tokenizer,
            seq_len=seq_len,
            filelist = filelist
        )
    
    def __iter__(self):
        """
            Yields (List[int])
        """
        ids = []
        for text in self._iterator:
            ids.extend(self._tokenizer.encode(text))
            while len(ids) >= self.seq_len+1:
                yield {"input_ids": ids[:self.seq_len],
                       "labels": ids[1:self.seq_len+1]}
                ids = ids[self.seq_len:]

    @classmethod
    def collate_fn(cls, item):
        """Collate function for DataLoader
        Args:
            item (List[dict[str, List[int]]])
        Returns:
            (dict[str, torch.Tensor]):
        """
        keys = item[0].keys()
        dic = {
            key: torch.tensor([x[key] for x in item])
            for key in keys
        }
        return dic
    
    def __len__(self):
        return len(self.filelist)

In [30]:
ds = FileListDataset.from_filelist(filelist=filelist[:2000], tokenizer=tokenizer, seq_len=512)

worker_info None


In [31]:
train_dataloader = DataLoader(ds, batch_size=4,  num_workers=2)

## in two processes it is ok

In [32]:
from tqdm import tqdm
epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=True)
for step, batch in enumerate(epoch_iterator):
    print ("\n\n\n")
    print (batch)
    print ("\n\n\n")
    inputs, labels = (batch["input_ids"], batch["input_ids"])
    print (len(batch["input_ids"]))
    print ("\n\n")
    print (batch["input_ids"])

worker_info it WorkerInfo(id=0, num_workers=2, seed=3253111887137394400, dataset=<__main__.FileListDataset object at 0x7f1623d3fe50>)
multiproc
worker id 0
self.fileidx  /notebook/greenAI/owt_files/openwebtext/texts/0428173-579fc7f012afd03af82af8b8170856e9.txt
shift 2
next /notebook/greenAI/owt_files/openwebtext/texts/0904320-0f10dbe2c01a48933369dd1988de75d5.txt



worker_info it WorkerInfo(id=1, num_workers=2, seed=3253111887137394401, dataset=<__main__.FileListDataset object at 0x7f1623d3fe50>)
multiproc
worker id 1
self.fileidx  /notebook/greenAI/owt_files/openwebtext/texts/0093470-c5a689c4d45072b45f1059bac6f6fd71.txt
shift 2
next /notebook/greenAI/owt_files/openwebtext/texts/0083163-00672c0b75d247586716007a720352b4.txt







{'input_ids': [tensor([5189,  284,  290, 1998]), tensor([  262,  7045, 10625,    11]), tensor([4153,  287,   13,  379]), tensor([11983, 36498,    18,  1551]), tensor([7027,  290,  960, 9305]), tensor([ 379, 1973,   35,  617]), tensor([262, 262, 680, 636]), ten

KeyboardInterrupt: 

Process Process-1:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/opt/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/opt/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/util.py", line 325, in _exit_function
    _run_finalizers()
  File "/opt/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/util.py", line 325, in _exit_function
    _run_finalizers()
  File "/opt/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/util.py", line 265, in _run_finalizers
    finalizer()
  File "/opt/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/util.py", line 265, in _run_finalizers
    finalizer()
  File "/opt/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/util.py", line 189, in __call__
    res = self._callback(*self._args, **self._kwa

In [ ]:
torch.utils.data.DataLoader(ds, num_workers=0)

In [19]:
len(train_filelist), len(val_filelist)

(98000, 1000)

In [5]:
import sys
sys.path.append("/notebook/greenAI/transformers/src")

In [8]:
from src.data_classes.iterable_dataset_mp import getListOfFiles, FileListDataset

In [11]:
filelist = getListOfFiles('/notebook/greenAI/owt_files/openwebtext/texts')
ds = FileListDataset.from_filelist(filelist=filelist[:200], tokenizer=tokenizer, seq_len=512)

worker_info None


In [ ]:
train_dataloader = DataLoader(ds, batch_size=4,  num_workers=2)

### Perplexity before fine-tuning

In [56]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

filelist = getListOfFiles("/notebook/greenAI/owt_files/openwebtext")
len(filelist)

21006

In [19]:
# model.eval()

In [29]:
nlls = []
for batch_idx, sample in tqdm(enumerate(val_dataloader, start=1)):
    # print(batch_idx, sample['input_ids'].shape)
    input_ids, label_ids = sample["input_ids"], sample["labels"]
    input_ids = input_ids.to(device=device)
    label_ids = label_ids.to(device=device)
    with torch.no_grad():
        # average(on number of tokens) negative log-likelihood for each token is returned as the loss
        outputs = model(input_ids, labels=label_ids)
        nlls.append(outputs['loss'])
print(torch.stack(nlls).sum() / batch_idx)
ppl = torch.exp(torch.stack(nlls).sum() / batch_idx)
ppl, torch.stack(nlls).sum() / batch_idx

384it [02:27,  2.60it/s]


KeyboardInterrupt: 

## Examples before fine-tuning

In [ ]:
model.eval()
i = 0
with torch.no_grad():
    for batch_idx, sample in tqdm(enumerate(val_dataloader, start=1)):
        # print(batch_idx, sample['input_ids'].shape)
        input_ids, label_ids = sample["input_ids"], sample["labels"]
        input_ids = input_ids.to(device=device)
        label_ids = label_ids.to(device=device)
        # average(on number of tokens) negative log-likelihood for each token is returned as the loss
        outputs = model(input_ids)
        print(tokenizer.decode(input_ids[0][-10:]))
        print(tokenizer.decode(label_ids[0][-1:]))
        print(tokenizer.decode(outputs.logits.argmax(dim=-1)[0][-1:]))
        print()
        i += 1
        if i > 50: break

In [ ]:
model.eval()
i = 0
with torch.no_grad():
    for batch_idx, sample in tqdm(enumerate(val_dataloader, start=1)):
        # print(batch_idx, sample['input_ids'].shape)
        input_ids, label_ids = sample["input_ids"], sample["labels"]
        input_ids = input_ids.to(device=device)
        label_ids = label_ids.to(device=device)
        # average(on number of tokens) negative log-likelihood for each token is returned as the loss
        outputs = model(input_ids)
        print(tokenizer.decode(input_ids[0][-10:]))
        print(tokenizer.decode(label_ids[0][-1:]))
        print(tokenizer.decode(outputs.logits.argmax(dim=-1)[0][-1:]))
        i += 1
        if i > 50: break

In [134]:
# loss_fn = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.eos_token_id)
# test_loss = []
# for batch_idx, sample in tqdm(enumerate(val_dataloader, start=1)):
#     print(batch_idx, sample['input_ids'].shape)
#     input_ids, label_ids = sample["input_ids"], sample["labels"]
#     input_ids = input_ids.to(device=device)
#     label_ids = label_ids.to(device=device)
#     with torch.no_grad():
#         # average negative log-likelihood for each token is returned as the loss
#         outputs = model(input_ids)
#         logits = outputs.logits  # shape: (batch_size, input_len, vocab_size)
#         loss = loss_fn(
#             input=logits.view(-1, logits.shape[-1]),
#             target=label_ids.view(-1)
#         )
#         test_loss.append(loss)
# test_loss, sum(test_loss) / batch_idx

# Train

In [22]:
for batch_idx, _ in tqdm(enumerate(train_dataloader, start=1)):
    pass
num_train_batches = batch_idx
num_train_batches

179829it [1:29:39, 33.43it/s]


179829

In [24]:
# 179829
num_train_batches

179829

In [20]:
!nvidia-smi

Mon May 16 18:05:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.129      Driver Version: 410.129      CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:BE:00.0 Off |                    0 |
| N/A   35C    P0    65W / 350W |   1872MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [21]:
# cross_entropy = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.eos_token_id)
# training_steps = num_train_batches // wandb.config.gradient_accumulation_steps * wandb.config.epochs
# optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.lr)
# scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=wandb.config.warmup_steps, num_training_steps=training_steps)
# train_losses = []
# val_losses = []

# for epoch in tqdm(range(1, wandb.config.epochs+1)):
    
#     model.train()
#     train_loss = 0
#     for train_batch_idx, item in enumerate(train_dataloader, start=1):
#         src, tgt = item["input_ids"], item["labels"]
#         src = src.to(device=device)
#         tgt = tgt.to(device=device)

#         # output = model(input_ids=src, labels=tgt)
#         if wandb.config.loss == 'cross-entropy':
#             output = model(input_ids=src)
#             logits = output.logits
#             loss = cross_entropy(
#                 input=logits.view(-1, logits.shape[-1]),
#                 target=tgt.view(-1)
#             )
#         elif wandb.config.loss == 'nll':
#             output = model(input_ids=src, labels=tgt)
#             loss = output.loss
#         else:
#             raise NotImplementedError('No such loss type: {}'.format(wandb.config.loss))
#         train_loss += loss.item() # before gradient accumulation step
#         wandb.log({'train_batch_loss': loss.item()})
#         if wandb.config.gradient_accumulation_steps > 1:
#             loss = loss / wandb.config.gradient_accumulation_steps
#         loss.backward() # accumulate gradients
            
#         if train_batch_idx % wandb.config.gradient_accumulation_steps == 0:
#             optimizer.step()
#             scheduler.step()  # Update learning rate schedule
#             optimizer.zero_grad()
            
#     # train_losses.append(train_loss / train_batch_idx)
#     # print(f'Epoch {epoch}, Average {train_batch_idx}-batch Train Loss: {train_loss / train_batch_idx}')
    
#     optimizer.zero_grad() # in case there are unused accumulated gradients
        
#     model.eval()
#     with torch.no_grad():
#         val_ce = 0
#         val_nll = 0
#         for val_batch_idx, item in enumerate(val_dataloader, start=1):
#             src, tgt = item["input_ids"], item["labels"]
#             src = src.to(device=device)
#             tgt = tgt.to(device=device)
        
#             output = model(input_ids=src, labels=tgt)
#             val_nll += output.loss.item()
#             logits = output.logits  # shape: (batch_size, input_len, vocab_size)
#             celoss = cross_entropy(
#                 input=logits.view(-1, logits.shape[-1]),
#                 target=tgt.view(-1)
#             )
#             val_nll += output.loss.item()
#             val_ce += celoss.item()
#     val_losses.append(val_ce / val_batch_idx)
            
#     # print(f'Epoch {epoch}, Average {val_batch_idx}-batch Val Loss: {val_loss / val_batch_idx}')
#     wandb.log({'train_epoch_loss': train_loss / train_batch_idx, 
#                'val_ce_loss': val_ce / val_batch_idx, 
#                'val_nll_loss': val_nll / val_batch_idx,
#                'lr': scheduler.get_lr()[0]})
    
#     if epoch % wandb.config.save_steps == 0:
#         torch.save(model.state_dict(), os.path.join(wandb.run.dir, f'checkpoint_{epoch:03}_{val_ce/val_batch_idx :.3f}_{val_nll/val_batch_idx :.3f}.pth'))
        
# wandb.save(os.path.join(wandb.run.dir, "checkpoint*"))

  0%|          | 0/50 [04:15<?, ?it/s]


KeyboardInterrupt: 

In [21]:
num_train_batches = 179829

In [ ]:
cross_entropy = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.eos_token_id)
training_steps = num_train_batches // wandb.config.gradient_accumulation_steps * wandb.config.epochs
optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.lr)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=wandb.config.warmup_steps, num_training_steps=training_steps)

for epoch in tqdm(range(1, wandb.config.epochs+1)):
    
    train_loss = 0
    for train_batch_idx, item in enumerate(train_dataloader, start=1):
        optimizer.zero_grad() # in case there are unused accumulated gradients
        
        model.train()
        src, tgt = item["input_ids"], item["labels"]
        src = src.to(device=device)
        tgt = tgt.to(device=device)

        # output = model(input_ids=src, labels=tgt)
        if wandb.config.loss == 'cross-entropy':
            output = model(input_ids=src)
            logits = output.logits
            loss = cross_entropy(
                input=logits.view(-1, logits.shape[-1]),
                target=tgt.view(-1)
            )
        elif wandb.config.loss == 'nll':
            output = model(input_ids=src, labels=tgt)
            loss = output.loss
        else:
            raise NotImplementedError('No such loss type: {}'.format(wandb.config.loss))
        train_loss += loss.item() # before gradient accumulation step
        wandb.log({'train_batch_loss': loss.item()})
        if wandb.config.gradient_accumulation_steps > 1:
            loss = loss / wandb.config.gradient_accumulation_steps
        loss.backward() # accumulate gradients
            
        if train_batch_idx % wandb.config.gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            optimizer.zero_grad()
            wandb.log({'lr': scheduler.get_lr()[0]})
            
        if train_batch_idx % 1000 == 0:
            model.eval()
            with torch.no_grad():
                val_ce = 0
                val_nll = 0
                for val_batch_idx, item in enumerate(val_dataloader, start=1):
                    src, tgt = item["input_ids"], item["labels"]
                    src = src.to(device=device)
                    tgt = tgt.to(device=device)

                    output = model(input_ids=src, labels=tgt)
                    val_nll += output.loss.item()
                    logits = output.logits  # shape: (batch_size, input_len, vocab_size)
                    celoss = cross_entropy(
                        input=logits.view(-1, logits.shape[-1]),
                        target=tgt.view(-1)
                    )
                    val_nll += output.loss.item()
                    val_ce += celoss.item()
                    
            wandb.log({'val_ce_loss': val_ce / val_batch_idx,
                       'val_nll_loss': val_nll / val_batch_idx})
            
        if train_batch_idx % wandb.config.save_steps == 0:
            torch.save(model.state_dict(), os.path.join(wandb.run.dir, f'checkpoint_{train_batch_idx:06}_{val_ce/val_batch_idx :.3f}_{val_nll/val_batch_idx :.3f}.pth'))

    # print(f'Epoch {epoch}, Average {val_batch_idx}-batch Val Loss: {val_loss / val_batch_idx}')
    wandb.log({'train_epoch_loss': train_loss / train_batch_idx})
    
        
wandb.save(os.path.join(wandb.run.dir, "checkpoint*"))

  0%|          | 0/1000 [00:00<?, ?it/s]/home/user/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


In [27]:
wandb.config.save_steps

500000

In [26]:
train_batch_idx, val_batch_idx

(118338, 612)

In [25]:
wandb.save(os.path.join(wandb.run.dir, "checkpoint*"))

wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


['/home/jovyan/notebooks/wandb/run-20220516_180448-12z3yzi5/files/checkpoint_118338_7.325_10.102.pth',
 '/home/jovyan/notebooks/wandb/run-20220516_180448-12z3yzi5/files/checkpoint_118338_7.325_10.102.pth']

In [24]:
torch.save(model.state_dict(), os.path.join(wandb.run.dir, f'checkpoint_{train_batch_idx:06}_{val_ce/val_batch_idx :.3f}_{val_nll/val_batch_idx :.3f}.pth'))

In [27]:
wandb.finish()

lr,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_batch_loss,█▇▅▅▅▄▄▄▄▄▂▃▃▃▃▃▂▂▂▂▃▂▂▂▃▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁
train_epoch_loss,█▇▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val_ce_loss,▁▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████
val_nll_loss,█▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
lr,5e-05
train_batch_loss,4.15632
train_epoch_loss,4.53707
val_ce_loss,6.41902
val_nll_loss,9.41599


In [24]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Wed May 11 16:50:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.129      Driver Version: 410.129      CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:57:00.0 Off |                    0 |
| N/A   30C    P0    64W / 350W |  31936MiB / 32480MiB |      0%      Default |
+---

# Perplexity after fine-tuning

In [28]:
nlls = []
for batch_idx, sample in tqdm(enumerate(val_dataloader, start=1)):
    # print(batch_idx, sample['input_ids'].shape)
    input_ids, label_ids = sample["input_ids"], sample["labels"]
    input_ids = input_ids.to(device=device)
    label_ids = label_ids.to(device=device)
    with torch.no_grad():
        # average(on number of tokens) negative log-likelihood for each token is returned as the loss
        outputs = model(input_ids, labels=label_ids)
        nlls.append(outputs['loss'])
print(torch.stack(nlls).sum() / batch_idx)
ppl = torch.exp(torch.stack(nlls).sum() / batch_idx)
ppl, torch.stack(nlls).sum() / batch_idx

45it [00:16,  2.65it/s]


tensor(4.7080, device='cuda:0')


(tensor(110.8295, device='cuda:0'), tensor(4.7080, device='cuda:0'))

# Examples after finetuning

In [10]:
model.load_state_dict(torch.load('wandb/run-20220512_152855-34ptf261/files/checkpoint_001_7.052_8.374.pth', map_location=device))

<All keys matched successfully>

In [30]:
model.eval()
with torch.no_grad():
    for batch_idx, sample in tqdm(enumerate(val_dataloader, start=1)):
        # print(batch_idx, sample['input_ids'].shape)
        input_ids, label_ids = sample["input_ids"], sample["labels"]
        input_ids = input_ids.to(device=device)
        label_ids = label_ids.to(device=device)
        # average(on number of tokens) negative log-likelihood for each token is returned as the loss
        outputs = model(input_ids)
        print(tokenizer.decode(input_ids[0][-10:]))
        print(tokenizer.decode(label_ids[0][-1:]))
        print(tokenizer.decode(outputs.logits.argmax(dim=-1)[0][-1:]))

1it [00:00,  2.15it/s]

 of propagation of the signals in the circuits.The
 structure
 of


2it [00:00,  2.26it/s]

% pure. Aliquots werestored at
 280
°


3it [00:01,  2.27it/s]

...

thalyl group was evidenced in thespectrum of
 sample
 19


678it [05:03,  2.23it/s]

 disparity’, Proc. SignalProcessing �
�
ett


679it [05:04,  2.23it/s]

, alone and incombination, there was a
 significant
 in


In [29]:
model.eval()
with torch.no_grad():
    for batch_idx, sample in tqdm(enumerate(val_dataloader, start=1)):
        # print(batch_idx, sample['input_ids'].shape)
        input_ids, label_ids = sample["input_ids"], sample["labels"]
        input_ids = input_ids.to(device=device)
        label_ids = label_ids.to(device=device)
        # average(on number of tokens) negative log-likelihood for each token is returned as the loss
        outputs = model(input_ids)
        print(tokenizer.decode(input_ids[0][-10:]))
        print(tokenizer.decode(label_ids[0][-1:]))
        print(tokenizer.decode(outputs.logits.argmax(dim=-1)[0][-1:]))

1it [00:00,  2.11it/s]

 different reasons.Two of them were too young to
 be
 the


2it [00:00,  2.17it/s]

 e t al.. Nat. Genet. 7
.
,


3it [00:01,  2.23it/s]

...

 additional interest is the recent mapping of thePAX
4
 gene


44it [00:19,  2.24it/s]

 et al., 1994]. Finally, comparative genomic hybrid
ization
 with


45it [00:19,  2.27it/s]

 Type:Map. GDB IDs: G00
-
G


In [81]:
# fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,5))

# ax1.set_title('Train Loss')
# ax1.set_xlabel('Epoch')
# ax1.plot(train_losses)

# ax2.set_title('Test Loss')
# ax2.set_xlabel('Epoch')
# ax2.plot(val_losses)

# plt.show()

In [16]:
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# model.train()

# losses = []
# for epoch in range(1, 2):
#     for train_batch_idx, item in enumerate(train_dataloader, start=1):
#         src, tgt = item["input_ids"], item["labels"]
#         src = src.to(device=device)
#         tgt = tgt.to(device=device)

#         outputs = model(input_ids=src, labels=tgt)
#         loss = outputs[0]
#         loss.backward()
#         optimizer.zero_grad()
#         losses.append(loss.item())
#         if train_batch_idx % 10 == 0:
#             print(train_batch_idx, loss.item())

10 9.014542579650879


In [21]:
!nvidia-smi

Mon May  9 16:27:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.129      Driver Version: 410.129      CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:57:00.0 Off |                    0 |
| N/A   30C    P0    65W / 350W |  27078MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  